In [67]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from flask import Flask, request, abort
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import datetime
import numpy as np


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [25]:
res = requests.Session()


In [26]:
def getPageNumber(content):
    startIndex = content.find('index') #找尋index起始位置
    endIndex = content.find('.html')   #找尋.html起始位置
    pageNumber = content[startIndex + 5: endIndex]
    #print(content)                    #傳入的參數    
    #print(int(pageNumber) + 1)        #取得到的頁數+1(為最新的一頁)
    return int(pageNumber) + 1         #回傳最新一頁

In [74]:
# hotArticle = []                   #將熱門文章串列清空以便每次重取

In [81]:
def crawPtt(plate):
    hotArticle = []                   #將熱門文章串列清空以便每次重取
    rs = requests.session()            #幫助我們把這一次的request都算在同一個session裡，這樣我們第二次對登入頁面發request時，csrfmiddlewaretoken value才不會又重新產生。
    load = {
        'from': '/bbs/'+plate+'/index.html',
        'yes': 'yes'
    }
    # res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data=load) #用post方式過入口驗證
    res=rs.get('https://www.ptt.cc/bbs/KoreaDrama/index.html')
    soup = BeautifulSoup(res.text, 'html.parser')   #用beautifulSoup解析HTML代碼並存入變數“soup”中
    allPageURL = soup.select('.btn.wide')[1]['href']#取得最新頁面的上一頁網址 例如: /bbs/joke/index6148.html
    print(allPageURL)
    startPage = getPageNumber(allPageURL)          #呼叫函式將/bbs/joke/index6148.html字串處理成6149
    indexList = []                                   #將網址串列清空                      
    for page in range(startPage, startPage - 83, -1):#將最新一頁往前82頁都放入搜尋網址串列
        pageURL = 'https://www.ptt.cc/bbs/'+plate+'/index{}.html'.format(page)#爬取的版加上頁數形成的網址
        indexList.append(pageURL)                   #放入網址串列(9頁欲爬取的網址)
    content = ''   #內容清空
    count = 0      #計算文章數
    
    # 抓取 文章標題 網址 推文數
    while indexList:
        index = indexList.pop(0)  #取出最新一頁的網址
        res = rs.get(index, verify=False)
        if res.status_code != 200:#如網頁忙線中,則先將網頁加入 indexList 並休息0.5秒後再連接
            indexList.append(index)
             #print (u'error_URL:',index)
             #time.sleep(0.5)
        else:#順利連到網址則
            hotArticle += crawlHotActicle(res)
             #print (u'OK_URL:', index)
             #time.sleep(0.06)

    hotArticle = sorted(hotArticle, key=sortHot,reverse =True)#將文章以推文數排列
    return hotArticle
    # np.savetxt('hotArticle.csv', hotArticle, delimiter='|')
    # for article in hotArticle:#將文章依序印出
    #     if count == 10:       #僅列出前10筆
    #         return content
    #     data = '{} {}.{}\n人氣:{}\n{}\n\n'.format(article.get('date',None),count+1,article.get('title', None), article.get('hot', None), article.get('url_link', None))
    #     content += data
    #     count += 1
    # return content

In [76]:
def crawlHotActicle(res):
    soup = BeautifulSoup(res.text, 'html.parser')#用beautifulSoup解析HTML代碼並存入變數“soup”中
    articleList = []                             #文章串列清空
    for r_ent in soup.find_all(class_="r-ent"):  #找出這個網址內所有class_="r-ent"找出每篇文章(在這裡我們用class過濾，不過class是python的關鍵詞，這樣該怎麼辦？加個下劃線就可以)
        if r_ent.find(class_="hl f1") or r_ent.find(class_="hl f3"):#找出每篇文章後在看這篇文章的推文數(hl f1為爆,hl f3十位數)
            if (r_ent.find(class_="hl f1")):     #如果文章是爆
                hot = r_ent.find(class_="hl f1").text.strip()#取得推文"爆"
                try:
                    # 先得到每篇文章的url
                    link = r_ent.find('a')['href']
                    if link:
                        # 確定得到url再去抓 標題 以及 推文數
                        title = r_ent.find(class_="title").text.strip() #取得標題
                        date_ =r_ent.find(class_="date").text.strip()
                        if len(date_)<5:
                          date_= '0' + date_
                        url_link = 'https://www.ptt.cc' + link          #取得連結
                        articleList.append({ 
                            'url_link': url_link,                       #將連結放到串列中
                            'title': title,                             #將標題放在串列中
                            'hot' :  hot, 
                            'date' : date_,                                #推文數放到串列中
                        })
                    #print(url_link)
                except Exception as e:                                  #如果連不上這網址可能被刪文了
                    # print u'crawPage function error:',r_ent.find(class_="title").text.strip()
                    # print('本文已被刪除')
                    print('delete', e)
                #print(r_ent.find(class_="hl f1"))
            if (r_ent.find(class_="hl f3")):                            #如果文章十位數的推文
                hot = r_ent.find(class_="hl f3").text.strip()           #取得推文數
                #print(r_ent.find(class_="hl f3"))
                try:
                    # 先得到每篇文章的篇url
                    link = r_ent.find('a')['href']                     
                    if link:
                        # 確定得到url再去抓 標題 以及 推文數
                        title = r_ent.find(class_="title").text.strip() #取得標題
                        date_ =r_ent.find(class_="date").text.strip()
                        if len(date_)<5:
                          date_= '0' + date_
                        url_link = 'https://www.ptt.cc' + link          #取得連結
                        articleList.append({  
                            'url_link': url_link,                       #將連結放到串列中
                            'title': title,                             #將標題放在串列中
                            'hot' :  hot,
                            'date' : date_,                               #推文數放到串列中
                        })
                    #print(url_link)
                except Exception as e:                                  #如果連不上這網址可能被刪文了
                    # print u'crawPage function error:',r_ent.find(class_="title").text.strip()
                    # print('本文已被刪除')
                    print('delete', e)
                    
    # print(articleList)
    return articleList

In [77]:
def sortHot(content):
    return content['hot']


In [84]:
# print(crawPtt('KoreaDrama'))
# np.savetxt('hotArticle.csv', crawPtt('KoreaDrama'), delimiter='|')
ans=crawPtt('KoreaDrama')


/bbs/KoreaDrama/index2506.html


In [86]:
ans[0]

{'date': '06/22',
 'hot': '爆',
 'title': '[LIVE] tvN《某一天滅亡來到我家門前》EP14',
 'url_link': 'https://www.ptt.cc/bbs/KoreaDrama/M.1624361068.A.E78.html'}

In [87]:
type(ans)

list

In [88]:
import pandas as pd  

df = pd.DataFrame(ans) 
df.to_csv('hotArticle.csv') 
